<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

  Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.)

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Hierarchy
=========

## Component 

Like in VHDL and Verilog, you can define components that can be used to build a design hierarchy. However, in SpinalHDL, you don’t need to bind their ports at instantiation


In [ ]:
class Sub extends Component{
  val a = in UInt(8 bits)
  val b = out UInt()    
  b :=  a
}

In [ ]:
class Top extends Component{
  val a = in UInt(8 bits)
  val b = out UInt(8 bits) 
    
  val u_sub = new Sub 
  u_sub.a := a
  b := u_sub.b
}
showRtl(new Top) 

### Full adder

In [ ]:
class AdderCell extends Component {
  //Declaring all in/out in an io Bundle is probably a good practice
  val io = new Bundle {
    val a, b, cin = in Bool
    val sum, cout = out Bool
  }
  //Do some logic
  io.sum := io.a ^ io.b ^ io.cin
  io.cout := (io.a & io.b) | (io.a & io.cin) | (io.b & io.cin)
}

class Adder(width: Int) extends Component {
 
  //Create 2 AdderCell
  val cell0 = new AdderCell
  val cell1 = new AdderCell
  cell1.io.cin := cell0.io.cout   //Connect cout of cell0 to cin of cell1

  // Another example which create an array of ArrayCell
  val cellArray = Array.fill(width)(new AdderCell)
  cellArray(1).io.cin := cellArray(0).io.cout   //Connect cout of cell(0) to cin of cell(1)
 
}
showRtl(new Adder(8)) 

In [ ]:
Range(1,9).foldLeft(0)((a,b)=>{println(s"$a-->$b");b})

### Input / output definition
The syntax to define inputs and outputs is the following:

Syntax | Description | Return
-|-|-
in/out Bool | Create an input/output Bool | Bool
in/out Bits/UInt/SInt[(x bit)] | Create an input/output of the corresponding type | T
in/out(T) | For all other data types, you should add the brackets around it.<br> Sorry this is a Scala limitation. | T
master/slave(T) | This syntax is provided by the spinal.lib. T should extend IMasterSlave : <br>Some documentation is available here |T

In [ ]:
 class Top extends Component{
  val a = slave Flow(UInt(8 bits))
  val b = master Flow(UInt(8 bits)) 
  b << a //  b <> a also ok
}
showRtl(new Top) 

There are some rules to follow with component interconnection:

- Components can only read output and input signals of child components
- Components can read their own output port values (unlike VHDL)

### Jump wire（飞线）
If for some reason, you need to read signals from far away in the hierarchy (debug, temporal patch) 
you can do it by using the value returned by **`some.where.else.theSignal.pull()`**.

In [ ]:
class xxCtrl extends Component{
  val start   = in Bool()
  val end     = out Bool()
  val counter = Reg(UInt(8 bits)) init 0
  when(start){counter.clearAll}
  .otherwise{counter := counter + 1}
  end := counter === 255
}

class xxTop extends Component{
    val start = in Bool()    
    val xx = out UInt()
    
    val ctrl = new xxCtrl    
    ctrl.start := start
    
    xx :=  ctrl.counter.pull() //Jump wire auto through IO
}
showRtl(new xxTop) 

### Pruned signals
SpinalHDL will never Pruned signals with names, for those resones:

- Sometime they are used for debug purposes in the wave
- Sometime, they should be part of the usefull RTL, but the user forgot to connect something to realy make them usefull. If they are removed from the netlist that's realy confusing to the user, and kind of hard to trace back where the missing connection is
- It allow to design things without having the whole thing done and look at the wave already

In [ ]:
class TopLevel extends Component { 
  val notRemoved1 = UInt(8 bits)
  val notRemoved2 = UInt(8 bits) 
  Reg(UInt(8 bits)) init 0  //pruned signal without name without loads
}
showRtl(new TopLevel)

### Generic(VHDL) / Parameter(Verilog)
If you want to parameterize your component, you can give parameters to the constructor of the component as follows:

In [ ]:
class MyAdder(width: BitCount) extends Component {
  val io = new Bundle{
    val a,b    = in UInt(width)
    val result = out UInt(width)
  }
  io.result := io.a + io.b
}

showRtl(new MyAdder(8 bits))

I you have several parameters, it is a good practice to give a specific configuration class as follows:
```scala
case class MySocConfig(axiFrequency  : HertzNumber,
                       onChipRamSize : BigInt,
                       cpu           : RiscCoreConfig,
                       iCache        : InstructionCacheConfig)

class MySoc(config: MySocConfig) extends Component {
    ...
}
```

## Function
The ways you can use Scala functions to generate hardware are radically different than VHDL/Verilog for many reasons:

- You can instantiate registers, combinatorial logic and components inside them.

- You don’t have to play with process/@always that limit the scope of assignment of signals

- Everything is passed by reference, which allows easy manipulation.
For example you can give a bus to a function as an argument, then the function can internaly read/write to it.
You can also return a Component, a Bus, or anything else from scala and the scala world.

### Component whith function

In [ ]:
class Top extends Component{
  val a = in UInt(8 bits)
  val b = out UInt(8 bits) 
  val c = out UInt(8 bits) 
    
  def pass(x: UInt, n : Int) = {
      val ret = UInt(n bits)
          ret := x 
      ret 
  }
    
  def pass2(x: UInt) = {
      class Fix(n: Int) extends Component {
          val a = in UInt()
          val b = out  UInt() 
          b := pass(in(a), n)
      }
      val res = new Fix(x.getWidth)
      res.a := x
      res.b
  }
    b := pass(a,8)
    c := pass2(a)
}
showRtl(new Top) 

### RGB to gray
For example if you want to convert a Red/Green/Blue color into greyscale by using coefficients, you can use functions to apply them:

In [ ]:
class Top extends Component{
    // Input RGB color
val r, g, b = UInt(8 bits)

// Define a function to multiply a UInt by a scala Float value.
def coef(value: UInt, by: Float): UInt = (value * U((255*by).toInt, 8 bits) >> 8)

// Calculate the gray level
val gray = coef(r, 0.3f) + coef(g, 0.4f) + coef(b, 0.3f)
}
showRtl(new Top) 

### Valid Ready Payload bus
For instance if you define a simple Valid Ready Payload bus, you can then define some useful functions inside of it.

In [ ]:
case class MyBus(payloadWidth: Int) extends Bundle with IMasterSlave {
  val valid   = Bool
  val ready   = Bool
  val payload = Bits(payloadWidth bits)

  // define the direction of the data in a master mode
  override def asMaster(): Unit = {
    out(valid, payload)
    in(ready)
  }

  // Connect that to this
  def <<(that: MyBus): Unit = {
    this.valid   := that.valid
    that.ready   := this.ready
    this.payload := that.payload
  }

  // Connect this to the FIFO input, return the fifo output
  def queue(size: Int): MyBus = {
    val fifo = new MyBusFifo(payloadWidth, size)
    fifo.io.push << this
    return fifo.io.pop
  }
}

class MyBusFifo(payloadWidth: Int, depth: Int) extends Component {
  val io = new Bundle {
    val push = slave(MyBus(payloadWidth))
    val pop  = master(MyBus(payloadWidth))
  }
 io.pop <> io.push
}

class Top extends Component {
  val io = new Bundle {
    val idata = slave(MyBus(8))
    val odata  = master(MyBus(8))
  }
  io.odata << io.idata.queue(32)
}
showRtl(new Top)

## Area
Sometimes, creating a Component to define some logic is overkill because you:   

- Need to define all construction parameters and IO (verbosity, duplication)
- Split your code (more than needed)

For this kind of case you can use an Area to define a group of signals/logic.

In [ ]:
class UartCtrl extends Component {
  
  val timer = new Area {
    val counter = Reg(UInt(8 bit))
    val tick = counter === 0
    counter := counter - 1
    when(tick) {
      counter := 100
    }
  }

  val tickCounter = new Area {
    val value = Reg(UInt(3 bit))
    val reset = False
    when(timer.tick) {          // Refer to the tick from timer area
      value := value + 1
    }
    when(reset) {
      value := 0
    }
  }

  val stateMachine = new Area {
   
  }
}
showRtl(new UartCtrl)

In VHDL and Verilog, sometimes prefixes are used to separate variables into logical sections.  
It is suggested that you use `Area` instead of this in SpinalHDL.  

[ClockingArea](https://spinalhdl.github.io/SpinalDoc-RTD/SpinalHDL/Structuring/clock_domain.html#clock-domain) is a special kind of `Area` that allows you to define chunks of hardware which use a given `ClockDomain`